<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section_4/01_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 対話データの作成
訓練用の対話データを読み込み、適切な形に整えて保存します。

## Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

## 対話文の取得
雑談対話コーパス「projectnextnlp-chat-dialogue-corpus.zip」をダウンロードします。  
このデータのライセンスについては、以下に記載があります。  
https://sites.google.com/site/dialoguebreakdowndetection/chat-dialogue-corpus  

解凍したフォルダをGoogle ドライブにアップします。  
フォルダからjsonファイルを読み込み、対話文として成り立っている文章を取り出してリストに格納します。  



In [ ]:
import glob  # ファイルの取得に使用
import json  # jsonファイルの読み込みに使用

path = "/content/drive/My Drive/live_ai_data/projectnextnlp-chat-dialogue-corpus/json"  # フォルダの場所を指定

files = glob.glob(path + "/*/*.json")  # ファイルの一覧
dialogues = []  # 複数の対話文を格納するリスト
file_count= 0  # ファイル数のカウント
for file in files:
    with open(file, "r") as f:
        json_dic = json.load(f)
        dialogue = []  # 単一の対話
        for turn in json_dic["turns"]:
            annotations = turn["annotations"]  # 注釈
            speaker = turn["speaker"]  # 発言者
            utterance = turn["utterance"]  # 発言

            if speaker=="U":  # 発言者が人間であれば
                dialogue.append(utterance) 
            else:  # 発言者がシステムであれば
                is_wrong = False
                for annotation in annotations:
                    breakdown = annotation["breakdown"]  # 分類
                    if breakdown=="X":  # 1つでも不適切評価があれば
                        is_wrong = True
                        break
                if is_wrong:
                    dialogue.clear()  # 対話をリセット
                else:
                    dialogue.append(utterance)  # 不適切評価が無ければ対話に追加
            
            if len(dialogue) >= 2:  # 単一の会話が成立すれば
                dialogues.append(dialogue.copy())
                dialogue.pop(0)  # 最初の要素を削除

    file_count += 1
    if file_count%100 == 0:
        print("files:", file_count, "dialogues", len(dialogues))

print("files:", file_count, "dialogues", len(dialogues))

## 対話の表示
リストに格納された対話の、最初の100ペアを表示します。

In [ ]:
for dialogue in dialogues[:100]:
    print(dialogue)

## 対話データの保存
対話データをcsvファイルとしてGoogle Driveに保存します。

In [ ]:
import csv
from sklearn.model_selection import train_test_split

dialogues_train, dialogues_test =  train_test_split(dialogues, shuffle=True)
path = "/content/drive/My Drive/live_ai_data/"  # 保存場所

with open(path+"dialogues_train.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_train)

with open(path+"dialogues_test.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_test)